In [ ]:
import json
from geopy.geocoders import Nominatim

# Hàm kiểm tra địa lý bằng geopy
def locate(city):
    geolocator = Nominatim(user_agent="location_check")
    try:
        location = geolocator.geocode(city)
        return location is not None  # Trả về True nếu địa điểm hợp lệ
    except Exception as e:
        return False


# Hàm xóa khóa "class"
def delete_class(item):
    # Nếu là từ điển
    if isinstance(item, dict): 
        if "class" in item:  
            item.pop("class", None)
        for key, value in item.items():
            item[key] = delete_class(value)

    # Nếu là list
    elif isinstance(item, list):  
        return [delete_class(sub_item) for sub_item in item]
    return item  


# Hàm xử lý jason
def transform_data(data):

    output = []  
    more_info = {"header": "More Information", "sub_content": []}  
    location = {"header": "Location", "sub_content": []} 
    temp_by = None  # Biến lưu trữ mục chứa "By"

    for item in data:

        # Xóa class
        item = delete_class(item)

        # Bỏ link_text rỗng
        if "link_text" in item and not item["link_text"]:
            continue

        # Xóa href 
        if "href" in item:
            del item["href"]

        # xử lý phần có "All Rights Reserved"
        if "sub_content" in item and any("All Rights Reserved" in temp for temp in item["sub_content"]):
            output.append({"header": "Copyright", "sub_content": item["sub_content"]})
            continue

        # Xử lý các mục có chứa "By"
        if "sub_content" in item and "By" in item["sub_content"][0]:
            if temp_by:
                output.append(temp_by) # Nếu 'temp_py' đã có nhóm "By" thì thêm nó vào output và thêm nhóm "By" mới vào 
            temp_by = {"header": item["sub_content"][0], "sub_content": []}        
        elif "link_text" in item:  # Đưa các mục con vào nhóm "By"
            if temp_by and item["link_text"] not in ["Pricing", "About"]:
                temp_by["sub_content"].append(item["link_text"])

            # Nếu không, thêm vào danh sách "Morse Information"
            else:                
                more_info["sub_content"].append(item["link_text"])

        # Xử lý Location nếu 'sub_content' là thành phố
        elif "sub_content" in item and locate(item["sub_content"][0]):            
            location["sub_content"].extend(item["sub_content"])
        else:
            if temp_by:
                output.append(temp_by)
                temp_by = None
            output.append(item)

    if temp_by:
        output.append(temp_by)

    output.append(more_info)
    output.append(location)

    return output

# Đọc dữ liệu JSON từ tệp
with open("privacy_policy.json", "r", encoding="utf-8") as file:
    raw_data = json.load(file)

# Chuyển đổi dữ liệu
data_final = transform_data(raw_data)

# Lưu nội dung vào tệp JSON
output_filename = "cleaned_data.json" 
with open(output_filename, "w", encoding="utf-8") as file:
    json.dump(data_final, file, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được làm sạch và lưu vào tệp '{output_filename}'.")